In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
import json
from pymongo import MongoClient
from odo import odo
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
#import os
#chromedriver = "/home/allisonschlissel/notebooks/chromedriver"
#os.environ["webdriver.chrome.driver"] = chromedriver



#driver = webdriver.Chrome(chromedriver)
#driver.get("https://mail.google.com")

In [ ]:
lst = ['http://www.metmuseum.org/art/collection#!?material=Paintings&showOnly=withImage&offset=' + str(x) + '&pageSize=0&sortBy=Relevance&sortOrder=asc&perPage=20' for x in range(0, 12017, 20)]
for i in lst:
    print i

In [ ]:
for i in range(len(lst)):
    driver = webdriver.PhantomJS()  
    driver.implicitly_wait(1)
    driver.get(lst[i])
    html = driver.page_source
    driver.quit()

In [ ]:
my_links = []

soup = BeautifulSoup(html, 'html.parser')

for figure in soup.find_all('figure',class_="card__standard-image"):
    fig = figure.find('a')['href']
    if '/art/collection/search/' in fig:
        url = 'http://www.metmuseum.org' + fig
        my_links.append(url)
        print url

In [ ]:
soup_objs = []

for i in my_links:
    response = requests.get(i)
    page = response.text
    link_soup = BeautifulSoup(page)
    soup_objs.append(link_soup)
    
response.status_code

In [ ]:
from collections import defaultdict

client = MongoClient()
db = client.paintings

artwork5 = db.artwork5

#soup_objects is all your links
for i in lst: 
    for soup in soup_objs: 
        art_dict = defaultdict(list)
        dets = soup.findAll('dt', class_='collection-details__tombstone--label')
        for s in dets:
            art_dict[s.text].append(s.nextSibling.nextSibling.text)
        art_dict['title'] = soup.find('h1', class_='collection-details__object-title').encode('utf-8')
            #art_dict['artist'] = soup.find('div',class_='collection-details__tombstone').encode('utf-8')
        art_dict['gallery'] = soup.find('div', class_='collection-details__location').encode('utf-8')
        art_dict['description'] = soup.find('div', class_='collection-details__label').encode('utf-8')
        images = soup.findAll('img')
        for i in images:
            if 'ng-src' in str(i):
                art_dict['image']=i.encode('utf-8')
        artwork5.insert_one(art_dict)

In [ ]:
artwork5.count()

In [ ]:
artwork5.find_one()

In [ ]:
import pandas as pd

df = pd.DataFrame(list(artwork5.find()))

In [ ]:
df.head(10)

In [ ]:
df_list = df.iloc[0].tolist()

In [ ]:
df_list

In [ ]:
import numpy as np

df['description'] = df['description'].str.replace('<div class="collection-details__label">\n</div>', '')
df['description'] = df['description'].str.replace('<div class="collection-details__label">\r\n', '')
df['description'] = df['description'].str.replace('<br/><br/>', '')
df['description'] = df['description'].str.replace('\r\n    </div>', '')
df['gallery'] = df['gallery'].str.replace('<div class="collection-details__location">\n<svg class="icon">\n<use xlink:href="/dist/icons.svg#ico-marker" xmlns:xlink="http://www.w3.org/1999/xlink"></use>\n</svg>\r\n', '')
df['gallery'] = df['gallery'].str.replace('\r\n    </div>', '')
df['title'] = df['title'].str.replace('<h1 class="collection-details__object-title">', '')
df['title'] = df['title'].str.replace('</h1>', '')

In [ ]:
df_list = df.iloc[0].tolist()
df_list

In [ ]:
df_list2 = df.iloc[2].tolist()
df_list2

In [ ]:
df.head(2)

In [ ]:
df_new = df.drop(['Accession Number:', 'Credit Line:', 'Dimensions:', 'Rights and Reproduction:', '_id'], axis=1)
df_new.head(50)

In [ ]:
test = df_new['Artist:'][0]
test = str(test).split()
for i in test:
    if i.isdigit():
        print i

In [ ]:
test = df_new['Artist:'][0]
test = str(test).replace('[','').replace(']','').replace('(','').replace(')','')
test=test.split()
for i in test:
    left=i.split('\\')[0]
    right=i.split('\\')[0]

In [ ]:
df_new['Date:'].isnull().sum()

In [ ]:
df_new['Period:'].isnull().sum()

In [ ]:
df_new['Culture:'].isnull().sum()

In [ ]:
df_new['Geography:'].isnull().sum()

In [ ]:
df_new['Country:'] = df_new['Artist:'].str.extract(r"\(([A-Za-z]+)\)", expand=True)

In [ ]:
df_new.head()

In [ ]:
import pandas as pd
print pd.show_versions()